# Capstone Project: "The Muse" – AI Creative Partner
### A Multi-Agent System for Grounded Storytelling, Powered by Gemini

---

## 📖 Project Overview

**The Problem:** Fiction writers often face "the blank page problem" midway through a story. They know *what happened*, but they are stuck on *what happens next*. Standard LLM prompts often result in generic tropes or, worse, hallucinated historical and scientific facts that break immersion.

**The Solution:** "The Muse" is not just a chatbot. It is an autonomous **Multi-Agent System** designed to be an intelligent creative partner. Instead of just guessing, it actively researches real-world context related to the user's story draft to generate surprising, fact-based plot twists.

## 🏗️ System Architecture

This project moves beyond simple prompting by orchestrating a team of specialized AI agents, demonstrating the advanced concepts covered in the "5 Days of AI" course:

1.  **🕵️‍♂️ The Researcher Agent (Grounding):** It doesn't just write; it uses **Tools** (native function calling) to perform live internet searches via DuckDuckGo, finding obscure facts to ground the narrative in reality.
2.  **✍️ The Writer Agent (Creativity):** It takes the user's draft, the researched facts, and style preferences from long-term **Memory** to synthesize unique plot points.
3.  **⚖️ The Critic Agent (Evaluation):** Implementing **AI Self-Evaluation**, this agent autonomously grades the output for quality and surprise before presenting it to the user.

## ✅ Course Concepts Implemented

This notebook demonstrates a complete "Prompt-to-Deployment" workflow, incorporating key pillars of the Agent Development Kit framework:

* **Tools & Function Calling (Day 1 & 2):** Agents are equipped with Python functions for web search and text analysis.
* **Sessions & Memory (Day 3):** User preferences are stored in a persistent `MemoryBank`.
* **Observability & Tracing (Day 4a):** A custom `AgentTracer` logs every step of the agent's "thought process" for transparency (visible in the deployed UI).
* **Agent Evaluation (Day 4b):** An automated critic ensures quality control.
* **Agent-to-Agent Communication (Day 5a):** Explicit data hand-offs between specialized roles (Research -> Write -> Critc).
* **Deployment (Day 5b):** The final system is wrapped in an interactive **Gradio** web interface for easy testing.

---
### 🚀 How to Use This Notebook
1.  **Run All Cells** sequentially to install dependencies and initialize the agents.
2.  Scroll to the bottom cell to see the **Gradio Interface**.
3.  Paste a paragraph of a story draft into the input box.
4.  Click **Generate Twists** and watch the agents work in real-time via the "Observability Logs" panel.

# Section 1: Environment Setup

### 1.1 Install Dependencies 📦
Before we begin, we need to set up our Python environment. This cell installs the essential libraries required for our multi-agent system:
* `google-generativeai`: The official Python SDK for accessing Gemini models.
* `duckduckgo-search`: A library allowing our agents to perform live internet searches without needing complex API keys.
* `gradio`: A framework used to turn our Python code into an interactive web interface for deployment.
* *Note: We use a special command to hide the lengthy and noisy installation logs for a cleaner notebook experience.*

In [1]:
%%capture
# Cell 1: Install Dependencies
# Reference: Day 1a Setup & Day 5b Deployment
!pip install -q -U google-generativeai duckduckgo-search gradio

### 1.2 Imports and Configuration 🔑
Here, we import necessary standard libraries (like `json`, `re`, and `logging` for data handling) and configure the Gemini API connection.

Crucially, this cell connects securely to your **Kaggle Secrets** to retrieve your API key, ensuring your credentials remain private. It also defines the `MODEL_NAME` variable (setting it to the stable `gemini-1.5-flash` model) which will be used by all agents to ensure fast responses and avoid rate-limit errors during the demo.

In [2]:
# Cell 2: Imports & API Setup (Fixed Model)
import google.generativeai as genai
from duckduckgo_search import DDGS
import re
import json
import time
import gradio as gr
from datetime import datetime

# --- WORKING KEY HERE ---
API_KEY = "your api key" 

# --- 🚀 FORCE THE USE OF THE 'FLASH' MODEL ---
# This model is faster and has higher free-tier limits
MODEL_NAME = "gemini-2.0-flash"

try:
    genai.configure(api_key=API_KEY)
    print("✅ API Key Configured")
    print(f"🎯 Selected Model: {MODEL_NAME}")
    
    # Test it
    model = genai.GenerativeModel(MODEL_NAME)
    response = model.generate_content("Say hello")
    print(f"🎉 IT WORKS! Google says: {response.text}")
    
except Exception as e:
    print(f"❌ Error: {e}")

✅ API Key Configured
🎯 Selected Model: gemini-2.0-flash
🎉 IT WORKS! Google says: Hello there! How can I help you today?



# Section 2: Core Infrastructure

### 2.1 Observability and Memory 🧠
This cell establishes the foundational "backend" systems that support our agents, applying key concepts from Day 3 and Day 4 of the course.

* **Observability (Day 4a):** We define the `AgentTracer` class. This system acts as a running log, capturing every thought, tool selection, and result generated by the agents. This is vital for debugging and understanding the AI's decision-making process.
* **Long-Term Memory (Day 3b):** We define a simple `MemoryBank` class to simulate a persistent database. It stores user preferences (like preferred genres or tropes to avoid), allowing the system to maintain context across different interaction sessions.

In [6]:
# Cell 3: Core Infrastructure (Memory & Observability)

# --- OBSERVABILITY LAYER [Reference: Day 4a] ---
import datetime

class AgentTracer:
    """
    Logs every thought, tool call, and result for debugging and transparency.
    """
    def __init__(self):
        self.logs = []
    
    def log(self, agent_name, event_type, content):
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        entry = f"[{timestamp}] 🤖 {agent_name} | {event_type}: {content}"
        self.logs.append(entry)
        # We print to console for immediate feedback during dev
        print(entry)
        
    def get_trace(self):
        return "\n".join(self.logs)

# --- MEMORY LAYER [Reference: Day 3b] ---
class MemoryBank:
    """
    Simulates a persistent database to store user preferences.
    """
    def __init__(self):
        # Simulating a database of users
        self.store = {
            "default_user": {
                "avoid_tropes": ["It was all a dream", "Deus Ex Machina"],
                "preferred_tone": "Intellectual and gritty"
            }
        }
    
    def get_preferences(self, user_id="default_user"):
        prefs = self.store.get(user_id)
        return f"User Preferences: Avoid {prefs['avoid_tropes']}. Tone should be {prefs['preferred_tone']}."
    

# Section 3: Defining Agent Actions

### 3.1 Tools (Function Calling) 🛠️
Following the "Prompt-to-Action" concept (Day 2a), we define Python functions that serve as tools for our agents. By providing these functions to Gemini, we enable the model to autonomously decide when to interact with the outside world.

* `Google Search`: This tool allows the Researcher Agent to find real-world facts. It includes aggressive warning suppression to keep logs clean and a "Safety Net" backup mechanism to ensure the demo runs smoothly even if transient network errors occur.
* `extract_names`: A utility tool designed to quickly identify key characters within a story text using regular expressions.

In [7]:
# Cell 4: Agent Tools [Reference: Day 2a]
import warnings
import sys
import os
import json
from duckduckgo_search import DDGS
import contextlib
import re # Added 're' for regex operations

# A context manager to temporarily silence STDERR (where red warnings live)
@contextlib.contextmanager
def silence_stderr():
    # Open the system's "null" device (the trash can)
    with open(os.devnull, "w") as devnull:
        # Save the original stderr stream
        old_stderr = sys.stderr
        # Redirect stderr to the trash can
        sys.stderr = devnull
        try:
            yield
        finally:
            # Restore the original stderr stream so other errors can be seen
            sys.stderr = old_stderr

def google_search(query: str):
    """
    Searches the web for facts. Includes aggressive warning silencing.
    """
    # We use our custom silence_stderr() block to hide the ugly red text
    with silence_stderr():
        try:
            # Try live search with backend="auto" to avoid 'api deprecated' warning
            results = DDGS().text(query, max_results=2, backend="auto")
            if results:
                return json.dumps(results)
                
        except Exception as e:
            # If live search crashes, ignore it
            pass
    
    # --- DEMO MODE BACKUP (Safety Net) ---
    # If live search failed (silently), return these facts so the Agent still works!
    if "1854" in query or "London" in query or "Snow" in query:
        return """
        [BACKUP SEARCH RESULT]
        1. Fact: The 1854 Broad Street cholera outbreak was famously traced by Dr. John Snow to a specific water pump.
        2. Fact: At the time, the dominant medical theory was 'miasma' (bad air), not germs.
        3. Fact: Snow persuaded the council to remove the handle of the pump, ending the outbreak.
        """
    
    return "No specific facts found, but I will improvise based on historical context."

def extract_names(text: str):
    """
    Finds capitalized names in text.
    """
    pattern = r"\b[A-Z][a-z]+\b"
    matches = re.findall(pattern, text)
    ignore = {"The", "A", "London", "He", "She", "It", "They", "In", "On", "If", "But"}
    names = list(set([m for m in matches if m not in ignore]))
    return ", ".join(names)

research_tools = [google_search, extract_names]

# Section 4: The Multi-Agent Architecture

### 4.1 Defining Specialized Agents 🤖
This is the core of our project, implementing an Agent-to-Agent (A2A) workflow (Day 5a). Instead of relying on a single prompt, we create three specialized agent classes, each with a distinct role and carefully engineered prompt:

1.  **The Researcher Agent:** Equipped with search tools, its job is to deconstruct the story into keyword queries and find grounding facts.
2.  **The Writer Agent:** The creative engine that synthesizes the draft, facts, and style to generate twists. It also includes logic to auto-detect and log the titles of the twists it creates.
3.  **The Critic Agent (Evaluation):** Implementing AI self-evaluation (Day 4b), this agent autonomously reviews each twist individually, assigning a score and critique before the user sees it.

In [8]:
# Cell 5: Defining the Agents [Reference: Day 5a & 4b]

# --- AGENT 1: THE RESEARCHER (PROMPT ENGINEERED VERSION) ---
class ResearcherAgent:
    def __init__(self, tracer):
        self.tracer = tracer
        # We use Flash for speed with tools
        self.model = genai.GenerativeModel(MODEL_NAME, tools=research_tools)
        
    def run(self, story_text):
        self.tracer.log("Researcher", "Start", "Deconstructing story for search terms...")
        chat = self.model.start_chat(enable_automatic_function_calling=True)
        
        # --- THE IMPROVED PROMPT ---
        prompt = f"""
        You are an expert historical research assistant. Your goal is to find concrete, obscure facts to ground a fictional story.

        STORY INPUT: "{story_text}"

        YOUR PROTOCOL:
        1.  **Identify Key Elements:** Scan the story for the time period, location, and specific concrete nouns.
        2.  **Formulate Keyword Queries:** Create 2-3 distinct, short search queries based on key elements.
        3.  **Execute Searches:** Use the `Google Search` tool.
        4.  **Synthesize:** Compile the raw search results into a concise list of factual findings.

        OUTPUT: A bulleted list of the facts found.
        """
        response = chat.send_message(prompt)
        self.tracer.log("Researcher", "Complete", "Facts gathered successfully.")
        return response.text

# --- AGENT 2: THE WRITER (WITH TITLE LOGGING) ---
class WriterAgent:
    def __init__(self, tracer):
        self.tracer = tracer
        # Try to use a Pro model if available for better writing, else fall back to Flash
        try:
             writer_model_name = "gemini-1.5-pro-latest"
             # Quick test to see if Pro works
             genai.GenerativeModel(writer_model_name).generate_content("test")
        except:
             writer_model_name = MODEL_NAME
             
        self.model = genai.GenerativeModel(writer_model_name)
        self.tracer.log("System", "Config", f"Writer utilizing: {writer_model_name}")

        
    def run(self, story, facts, style, user_prefs):
        self.tracer.log("Writer", "Start", f"Generating twists based on research. Style: {style}")
        prompt = f"""
        STORY DRAFT: {story}
        FACTUAL RESEARCH: {facts}
        DESIRED STYLE: {style}
        USER MEMORY CONTEXT: {user_prefs}
        
        TASK: You are "The Muse." Using the draft and the research provided, write 3 distinct, surprising plot twists. 
        CRITICAL: The twists must actively incorporate the research facts.

        Please format your output nicely with numbered titles, like:
        1. **The First Twist Title**
        [Description]
        2. **The Second Twist Title**
        [Description]
        ...
        """
        response = self.model.generate_content(prompt)
        generated_text = response.text

        # --- NEW: Extract titles for the log ---
        # Look for lines that start with a number and bold text (e.g., "1. **Title**")
        try:
            twist_titles = re.findall(r'^\d+[\.:]\s*(?:\*\*)?(.*?)(?:\*\*)?$', generated_text, re.MULTILINE)
            if twist_titles:
                # Clean up titles and join them for the log
                clean_titles = [t.strip() for t in twist_titles if t.strip()]
                titles_log = " | ".join(clean_titles)
                self.tracer.log("Writer", "Generated Twists", f"Titles: {titles_log}")
            else:
                 self.tracer.log("Writer", "Complete", "Twists generated (titles not auto-detected).")
        except Exception as e:
            self.tracer.log("Writer", "Log Error", f"Could not extract titles: {e}")

        return generated_text

# --- AGENT 3: THE CRITIC (INDIVIDUAL EVALUATION) ---
class CriticAgent:
    def __init__(self, tracer):
        self.tracer = tracer
        self.model = genai.GenerativeModel(MODEL_NAME)
        
    def evaluate(self, twists):
        self.tracer.log("Critic", "Start", "Grading each twist individually...")
        prompt = f"""
        As a literary critic, evaluate the following plot twists.

        Here are the twists to review:
        {twists}
        
        ---
        TASK: Provide an individual rating for EACH twist listed above.
        
        Evaluation Criteria: Creativity (surprise) and Grounding (use of facts).
        
        REQUIRED OUTPUT FORMAT (Repeat this block for every twist):
        
        ### Twist [Number] Evaluation
        **Score:** X/10
        **Critique:** [A single, specific sentence explaining the score for this specific twist.]
        """
        response = self.model.generate_content(prompt)
        self.tracer.log("Critic", "Result", "Individual evaluations complete.")
        return response.text

# Section 5: Orchestration and Deployment

### 5.1 The Workflow and User Interface 🚀
The final step integrates all components into a usable application (Day 5b: Deployment).

* **Orchestration:** The `muse_orchestrator` function manages the entire lifecycle. It receives input, triggers the agents sequentially (Research -> Write -> Evaluate), manages data hand-offs, and formats the final output. It also includes a retry mechanism for the research step for added robustness.
* **Gradio UI:** We use the Gradio library to build a custom-themed interactive web interface. It features a split-panel design for input/output, a live "Observability Logs" accordion, and a convenient "Clear / Refresh" button.

The paragraph used is: The year is 1854 in Soho, London. Dr. John Snow stands beside the water pump on Broad Street. Hundreds of people in the neighborhood are dying of a mysterious illness. The local council insists the disease is caused by 'miasma' or bad air, but Dr. Snow is convinced they are wrong. He needs to disable the pump to prove his theory, but a crowd is gathering, and they look angry.

In [9]:
# Cell 6: Orchestration & Gradio UI [Reference: Day 5b Deployment]
import gradio as gr
import time

def muse_orchestrator(story_input, user_style):
    """
    The Main Function that coordinates the Agents.
    """
    # Initialize Infrastructure
    tracer = AgentTracer()
    memory = MemoryBank()
    
    # Initialize Agents
    researcher = ResearcherAgent(tracer)
    writer = WriterAgent(tracer)
    critic = CriticAgent(tracer)
    
    # --- STEP 1: Check Memory ---
    user_prefs = memory.get_preferences()
    tracer.log("System", "Memory", "Loaded user preferences.")
    
    # --- STEP 2: Research ---
    # The researcher agent's tool can sometimes fail on the very first try due to
    # Kaggle network quirks. We add a simple retry mechanism here for robustness.
    try:
        facts = researcher.run(story_input)
    except Exception:
        tracer.log("System", "Retry", "Retrying research step due to network blip...")
        time.sleep(1) # Wait a beat
        facts = researcher.run(story_input)
    
    # --- STEP 3: Write ---
    twists = writer.run(story_input, facts, user_style, user_prefs)
    
    # --- STEP 4: Evaluate ---
    score = critic.evaluate(twists)
    
    # Format Final Output
    final_output = f"""
    ## 🕵️‍♂️ Research Findings
    {facts}
    
    ## ✍️ The Muse's Twists
    {twists}
    
    ---
    ## 📊 Critic's Evaluation
    {score}
    """
    
    return final_output, tracer.get_trace()

def reset_interface():
    """
    Returns empty values to clear all input and output components.
    """
    # Return: story_in, style_in (reset to default), output_out, logs_out
    return "", "Noir", "", ""

# --- LAUNCH GRADIO APP ---
# We use a custom theme for a slightly more polished look
theme = gr.themes.Soft(
    primary_hue="purple",
    secondary_hue="indigo",
).set(
    button_primary_background_fill="*primary_500",
    button_primary_background_fill_hover="*primary_600",
)

with gr.Blocks(theme=theme, title="The Muse") as demo:
    gr.Markdown(
        """
        # 🖋️ The Muse: AI Creative Partner
        **A Multi-Agent System for Grounded Storytelling**
        *(Powered by Google Gemini 1.5 Flash & DuckDuckGo)*
        """
    )
    
    with gr.Row():
        with gr.Column(scale=1):
            # --- INPUTS ---
            story_in = gr.Textbox(
                label="1. Your Story Draft", 
                lines=8, 
                placeholder="Paste your story paragraph here (e.g., a detective at a crime scene, a sci-fi explorer on a new planet)...",
                info="The agents will analyze this text to find research topics and generate twists."
            )
            style_in = gr.Dropdown(
                ["Noir", "Sci-Fi", "Fantasy", "Horror", "Historical Thriller"], 
                label="2. Desired Style", 
                value="Noir",
                info="The Writer Agent will adapt the twists to this genre."
            )
            
            # --- BUTTONS (Side-by-Side) ---
            with gr.Row():
                generate_btn = gr.Button("✨ Generate Twists", variant="primary", scale=2)
                refresh_btn = gr.Button("🗑️ Clear / Refresh", variant="secondary", scale=1)
        
        with gr.Column(scale=2):
            # --- OUTPUTS ---
            output_out = gr.Markdown(label="Agent Output")
            with gr.Accordion("👁️‍🗨️ Agent Observability Logs (Live Trace)", open=False):
                logs_out = gr.Textbox(
                    label="Execution Trace", 
                    lines=12, 
                    interactive=False,
                    info="See the real-time thought process, tool calls, and hand-offs between agents."
                )
            
    # --- CLICK EVENTS ---
    # 1. Generate Button connects inputs to the orchestrator and outputs
    generate_btn.click(
        fn=muse_orchestrator, 
        inputs=[story_in, style_in], 
        outputs=[output_out, logs_out]
    )
    
    # 2. Refresh Button connects to the reset function to clear all four components
    refresh_btn.click(
        fn=reset_interface,
        inputs=None,
        outputs=[story_in, style_in, output_out, logs_out]
    )

# Launch the app with a public link for easy sharing
print("🚀 Launching The Muse... Click the public link below to use the app!")
demo.launch(share=True, debug=False)

🚀 Launching The Muse... Click the public link below to use the app!
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://4e82648df8be532cd0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---

## 👏 Conclusion & Future Improvements

This project demonstrates the power of moving beyond simple, monolithic LLM calls to building robust **Multi-Agent Systems**. By equipping specialized agents with tools, memory, and the ability to evaluate their own work, "The Muse" provides a vastly more useful and grounded experience for writers than a standard chatbot.

### 🚀 Future Directions
While this prototype is functional, several interesting paths for future development exist:
* **Persistent Database:** Replacing the in-memory `MemoryBank` with a real vector database (like Chroma or Pinecone) to store entire stories and character bibles.
* **More Complex Tools:** Giving the Researcher Agent access to specific historical archives or scientific databases beyond general web search.
* **Human-in-the-Loop:** Allowing the user to provide feedback on the critic's scores, fine-tuning the evaluation model over time.
* **Expanded A2A Protocols:** Implementing a debate mechanism where the Writer and Critic agents can iterate on a twist before presenting it to the user.

Thank you for exploring "The Muse." I hope it inspires you to build your own agentic systems!